In [2]:
# imports
import pandas as pd
import numpy as np

In [5]:
# Pull in the data
file = "../data/data.csv"
df_transactions = pd.read_csv(file)
df_transactions.head()

,Customer ID,Age,Gender,Transact ID,Transact Date,Transact Time,Outlet,Outlet District,Transact Details ID,Item,Item Description,Quantity,Price,Spending
0,57618,58.0,F,183003,1/1/16,12:01:48 AM,Outlet 17,5,537557,215A,Pork Chop Noodle,1,13.94,13.94
1,57618,58.0,F,183003,1/1/16,12:01:48 AM,Outlet 17,5,537555,34,Fish & Chips,1,13.94,13.94
2,54234,40.0,NaN,183005,1/1/16,12:03:32 AM,Outlet 11,27,537565,552,Mango Pomelo,1,8.36,8.36
3,54234,40.0,NaN,183005,1/1/16,12:03:32 AM,Outlet 11,27,537564,215B,Pork Chop Bee Hoon,1,13.94,13.94
4,73173,30.0,NaN,183010,1/1/16,12:14:51 AM,Outlet 14,16,537571,127,Borsch Soup,2,5.94,11.88


In [7]:
mean = 38
sd = 11
np.norm

df_transactions.ix[df_transactions["Age"] <= 5].

,Customer ID,Age,Gender,Transact ID,Transact Date,Transact Time,Outlet,Outlet District,Transact Details ID,Item,Item Description,Quantity,Price,Spending
93,54582,-182.0,NaN,183095,1/1/16,11:00:16 AM,Outlet 11,27,537806,902,Cent Egg Pork Por,1,10.30,10.30
94,54582,-182.0,NaN,183095,1/1/16,11:00:16 AM,Outlet 11,27,537804,713,Hot Coffee-Tea,1,4.24,4.24
95,54582,-182.0,NaN,183095,1/1/16,11:00:16 AM,Outlet 11,27,537805,715,Milk Tea,1,4.24,4.24
517,62739,-5943.0,NaN,184017,1/1/16,7:50:15 PM,Outlet 20,19,540762,797,RAINBOW SPRITE,1,5.94,5.94
518,62739,-5943.0,NaN,184017,1/1/16,7:50:15 PM,Outlet 20,19,540763,808,Chicken Dry Curry Ramen,1,13.94,13.94
519,62739,-5943.0,NaN,184017,1/1/16,7:50:15 PM,Outlet 20,19,540764,909,Sliced Empress Congee,1,12.00,12.00
1738,54426,-182.0,NaN,186250,3/1/16,6:16:37 PM,Outlet 11,27,547664,701,Chinese Tea,1,2.06,2.06
1739,54426,-182.0,NaN,186250,3/1/16,6:16:37 PM,Outlet 11,27,547666,776,Watermelon Ice,1,7.15,7.15
1740,54426,-182.0,NaN,186250,3/1/16,6:16:37 PM,Outlet 11,27,547662,305,F.Fries Curry Mayo,1,5.94,5.94
1741,54426,-182.0,NaN,186250,3/1/16,6:16:37 PM,Outlet 11,27,547660,121,Egg & Ham Sandwich,1,4.73,4.73


In [97]:
df_customer = pd.DataFrame()
df_customer["customerID"] = df_transactions["Customer ID"]
df_customer["age"] = df_transactions["Age"]
df_customer["gender"] = df_transactions["Gender"]

In [102]:
print("Customer table size ", len(df_customer.index))
df_customer = df_customer.drop_duplicates(keep='first')
print("Customer table size after removing duplicates ", len(df_customer.index))

Customer table size  556580
Customer table size after removing duplicates  44414


In [103]:
df_customer['age'].min()
df_customer['age'].max()

116.0

In [104]:
df_customer['age'] = pd.to_numeric(df_customer['age'])
df_customer['age'] = df_customer['age'].fillna(value=0)
df_customer['gender'] = df_customer['gender'].fillna(value="NA")

In [105]:
mean = df_customer["age"].ix[(df_customer["age"] >= 6) & (df_customer["age"] <= 88)].mean()
sd = df_customer["age"].ix[(df_customer["age"] >= 6) & (df_customer["age"] <= 88)].std()
print("Mean: ", mean, " SD: ", sd)

Mean:  37.3733306078  SD:  11.9079709467


In [109]:

def assignAge(d):
    while(d < 6 or d > 88):
        d = np.random.normal(mean, sd, 1)[0]
    return int(d)
        
df_customer["age"].apply(assignAge)

0         58
2         40
4         30
9         57
17        39
20        58
23        32
27        30
34        27
37        31
44        31
54        31
58        35
63        45
68        48
74        59
82        60
87        26
89        37
93        39
96        35
99        38
104       38
108       58
116       37
123       47
127       34
135       25
137       48
144       45
          ..
555998    26
556009    44
556050    44
556073    52
556075    50
556110    18
556124    34
556148    29
556187    35
556188    38
556192    53
556201    20
556203    22
556224    27
556227    24
556253    36
556294    58
556298    40
556299    22
556326    41
556335    63
556389    32
556399    21
556406    43
556415    18
556419    47
556450    42
556490    26
556525    24
556537    26
Name: age, dtype: int64

In [111]:
df_customer.ix[df_customer["customerID"] == 66515]

,customerID,age,gender
23291,66515,28.0,NA


In [110]:
np.random.normal(mean, sd, 1)[0]

30.320286207421908

In [ ]:
def age_group(d):
    if d <= 21:
        return "YOUTH"
    elif d <= 50 && d > 21:
        return "ADULT"
    else return "SENIOR"

df_customer['age_group'].apply()

In [113]:
df_transactions.groupby('Transact ID')['Spending'].sum()

Transact ID
183003     27.88
183005     22.30
183010     60.73
183017     71.03
183020     12.24
183028     29.69
183030     25.94
183032     41.33
183034     37.45
183048    152.85
183056     25.82
183058     70.78
183060     35.39
183062     20.85
183066     20.96
183070     37.81
183076     40.12
183082     25.94
183086      4.73
183092     16.61
183093     17.45
183095     18.78
183104     39.15
183113     14.91
183116     56.60
183127     37.58
183146     33.46
183152     86.66
183160     34.91
183163      5.94
           ...  
601782     55.76
601785     34.43
601789     29.94
601790     21.82
601791    196.47
601793     31.27
601795     19.39
601796     52.85
601800     62.18
601805     15.03
601815     34.91
601816     22.55
601817     50.79
601819     25.34
601821     40.86
601825      6.67
601827     23.89
601828     84.12
601830     21.45
601835     12.00
601837     41.45
601839      5.94
601842      9.82
601843     48.98
601849     11.88
601850     25.95
601852     37.69
60